# Mount your google drive

You can mount your google drive in colab in order to have access to permanent data storage.

In [1]:
import os
# mount your google drive
# you will need to authorize the access
from google.colab import drive
root = '/content/gdrive/'
drive.mount(root)

# this is the root file for your google drive
gdrive_root = os.path.join(root, 'My Drive')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# now, you can access and explore the contents of your gdrive:
gdrive_files = os.listdir(gdrive_root)
print(gdrive_files)

In [2]:
# make a folder to store results for this course
res_folder = os.path.join(gdrive_root, 'dl-course')
os.makedirs(res_folder, exist_ok=True)

## Save model checkpoints on your drive

When running long trainings it is very convenient to save the checkpoints on the drive in order to resume training if the gpu becomes unavailable or you lose connection to colab.

Below you find a small example that shows how to save checkpoints and load the checkpoint again to resume training.

In [3]:
# import the functionality from torch to train a small mnist model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

# save the mnist dataset on your drive
mnist_path = os.path.join(res_folder, 'mnist')
ds_train = MNIST(mnist_path, download=True, transform=ToTensor())

loader = DataLoader(ds_train, batch_size=8)
loss = nn.CrossEntropyLoss()

# the path were we will save the model and optimizer state on our drive
state_path = os.path.join(res_folder, 'model.pth')

# training for one epoch
def train_epoch(model, loader, loss, optimizer):
  for x, y in loader:
    x = x.view(-1, 28 * 28)
    pred = model(x)
    optimizer.zero_grad()
    loss_val = loss(pred, y)
    loss_val.backward()
    optimizer.step()

In [8]:
# train a linear model on mnist for 2 epochs and close it
# (don't mind the choice of model etc. here, this is just a small example to 
#  show how to save and load the checkpoints)

# linear layer for all pixels in an mnist image
model = torch.nn.Linear(28 * 28, 10)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(2):
  print("Training epoch", epoch, "...")
  train_epoch(model, loader, loss, optimizer)

# save the state of the model and optimizer to google drive
torch.save({'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch}, state_path)

Training epoch 0 ...
Training epoch 1 ...


In [4]:
# load the model again and continue training
# you can execute this without running the above cell and continue training from
# the saved checkpoint
model = torch.nn.Linear(28 * 28, 10)
optimizer = optim.SGD(model.parameters(), lr=0.001)

state = torch.load(state_path)
model.load_state_dict(state['model'])
optimizer.load_state_dict(state['optimizer'])

start_epoch = state['epoch'] + 1
for epoch in range(start_epoch, start_epoch + 2):
  print("Training epoch", epoch, "...")
  train_epoch(model, loader, loss, optimizer)

Training epoch 2 ...
Training epoch 3 ...
